# Importar y exportar modelos en formato ONNX

Vamos a usar el modelo corespondiente al propuesto en la práctica. Para ello, usaremos la librería `tf2onnx`, la continuación del proyecto `keras2onnx` que facilita enormemente la conversión entre formatos. Se instala de la siguiente manera.

```bash
$ pip install tf2onnx
```

Empezamos declarando las dependencias

In [1]:
import numpy as np
import onnx
import tensorflow as tf
import tf2onnx

from onnx2keras import onnx_to_keras

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

## Modelo de ejemplo

Vamos a crear un perceptrón multicapa simple con el que trabajar. Lo entrenaremos antes con datos aleatorios para que todos los pesos tengan valores

In [3]:
X = np.random.random((1000, 3))
y = np.eye(3)[np.random.choice(3, 1000)]

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_dim=3),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(3, activation='softmax'),
])

model.compile(
    loss='binary_crossentropy',
    optimizer='sgd',
    metrics=['accuracy'],
)

model.summary()

model.fit(X, y, epochs=100, verbose=0);

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               512       
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 387       
Total params: 899
Trainable params: 899
Non-trainable params: 0
_________________________________________________________________


## Conversión de modelo de Keras a ONNX


In [4]:
model_proto, _ = tf2onnx.convert.from_keras(
    model,
    input_signature=(tf.TensorSpec((None, 3), name='input'),),
    output_path='best-captcha-solver.onnx',
)

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


Y ya está, la verdad es que tampoco es mucho más complicado que esto.

## Carga de modelo ONNX a Keras

In [5]:
onnx_model = onnx.load('best-captcha-solver.onnx')
keras_model = onnx_to_keras(
    onnx_model, ['input'], 
    name_policy='short',
)
keras_model.compile(
    loss='binary_crossentropy',
    optimizer='sgd',
    metrics=['accuracy'],
)
keras_model.summary()

print(f'(loss, acc) of keras model before loading: {model.evaluate(X, y, verbose=0)}')
print(f'(loss, acc) of keras model after loading: {keras_model.evaluate(X, y, verbose=0)}')

INFO:onnx2keras:Converter is called.
DEBUG:onnx2keras:List input shapes:
DEBUG:onnx2keras:None
DEBUG:onnx2keras:List inputs:
DEBUG:onnx2keras:Input 0 -> input.
DEBUG:onnx2keras:List outputs:
DEBUG:onnx2keras:Output 0 -> dense_1.
DEBUG:onnx2keras:Gathering weights to dictionary.
DEBUG:onnx2keras:Found weight sequential/dense_1/MatMul/ReadVariableOp:0 with shape (128, 3).
DEBUG:onnx2keras:Found weight sequential/dense_1/BiasAdd/ReadVariableOp:0 with shape (3,).
DEBUG:onnx2keras:Found weight sequential/dense/MatMul/ReadVariableOp:0 with shape (3, 128).
DEBUG:onnx2keras:Found weight sequential/dense/BiasAdd/ReadVariableOp:0 with shape (128,).
DEBUG:onnx2keras:Found input input with shape [3]
DEBUG:onnx2keras:######
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Converting ONNX operation
DEBUG:onnx2keras:type: MatMul
DEBUG:onnx2keras:node_name: sequential/dense/MatMul:0
DEBUG:onnx2keras:node_params: {'change_ordering': False, 'name_policy': 'short'}
DEBUG:onnx2keras:...
DEBUG:onnx2keras:Check if all

Tensor("Placeholder:0", shape=(None, 128), dtype=float32) Tensor("Placeholder_1:0", shape=(128,), dtype=float32)
Tensor("Placeholder:0", shape=(None, 3), dtype=float32) Tensor("Placeholder_1:0", shape=(3,), dtype=float32)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input (InputLayer)              [(None, 3)]          0                                            
__________________________________________________________________________________________________
sequenti (Dense)                (None, 128)          384         input[0][0]                      
__________________________________________________________________________________________________
sequenti_1_const2 (Lambda)      (128,)               0           input[0][0]                      
______________________________________________________________________

Dos notas a tener en cuenta sobre el estado de ONNX en la actualidad:

- Los modelos salvados son optimizados. Esto implica que se realizan operaciones que **pueden alterar** los pesos de las conexiones (ligeramente, pero es un hecho)
- No funcionará con todas las capas. Por ejemplo, las LSTM Bidireccionales aún no están soportadas

En definitiva, es un estándar que está creciendo pero al que le faltan muchos detalles por apurar.